<a href="https://colab.research.google.com/github/felixsimard/comp551-p3/blob/main/Hassan_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Notebook to build CNN image classifier
- Based on https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#training-on-gpu

In [1]:
import numpy as np
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
from google.colab import drive

## Loading image data into np.ndarray

In [2]:
# Felix's load data fn
# Function to return pickle loaded file in an ndarray
def load_data(filename, data_path='/content/drive/MyDrive/data/'):
    drive.mount("/content/drive")
    loaded_pkl = None
    try:
        pkl_buffered = open(data_path+''+filename,'rb')
        loaded_pkl = pickle.load(pkl_buffered)
    except Exception as e:
        print("Error loading data: {}".format(e))
    return loaded_pkl

In [3]:
# loading all data
train_features = load_data("images_l.pkl")
train_labels = load_data("labels_l.pkl")
test = load_data("images_test.pkl")
train_unlabelled = load_data("images_ul.pkl")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
print(train_labels.shape, train_features.shape)

(30000, 36) (30000, 56, 56)


- `train_features` has 30,000 samples of 56x56 images
- `train_labels` labels of the 56x56 images, a 36-bit binary vector
- The code block below verifies the image data are all in numpy n-dimensional arrays, `np.ndarray`

In [10]:
for data in [train_features, train_labels, train_unlabelled, test]:  
  print(type(data) is np.ndarray)

True
True
True
True


## Building the CNN

In [11]:
class CNN(nn.Module):

  def __init__(self):
    super().__init__()

  def forward(self, x):
    return x
  